In [54]:
import json
import pennylane as qml
import pennylane.numpy as np

num_wires = 4
dev = qml.device("default.mixed", wires=num_wires)

@qml.qnode(dev)
def heisenberg_trotter(couplings, p, time, depth):
    """This QNode returns the final state of the spin chain after evolution for a time t, 
    under the Trotter approximation of the exponential of the Heisenberg Hamiltonian.
    
    Args:
        couplings (list(float)): 
            An array of length 4 that contains the coupling constants and the magnetic field 
            strength, in the order [J_x, J_y, J_z, h].
        p (float): The depolarization probability after each CNOT gate.
        depth (int): The Trotterization depth.
        time (float): Time during which the state evolves

    Returns:
        (numpy.tensor): The evolved quantum state.
    """
    #for i in range

    dt=time/depth
    Jx,Jy,Jz,h=couplings
    for k in range(depth):
        for i in range(4):
            q1=i
            q2=(i+1)%4
            qml.RY(np.pi/2,wires=[q1])
            qml.RY(np.pi/2,wires=[q2])
            qml.CNOT(wires=[q1,q2])
            qml.DepolarizingChannel(p, wires=q2)
            qml.RZ(-2*Jx*dt,wires=[q2])
            qml.CNOT(wires=[q1,q2])
            qml.DepolarizingChannel(p, wires=q2)
            qml.RY(-np.pi/2,wires=[q1])
            qml.RY(-np.pi/2,wires=[q2])
            #qml.IsingXX(-2*Jx*dt,wires=[q1,q2])
        for i in range(4):
            q1=i
            q2=(i+1)%4
            qml.RX(np.pi/2,wires=[q1])
            qml.RX(np.pi/2,wires=[q2])
            qml.CNOT(wires=[q1,q2])
            qml.DepolarizingChannel(p, wires=q2)
            qml.RZ(-2*Jy*dt,wires=[q2])
            qml.CNOT(wires=[q1,q2])
            qml.DepolarizingChannel(p, wires=q2)
            qml.RX(-np.pi/2,wires=[q1])
            qml.RX(-np.pi/2,wires=[q2])
            #qml.IsingYY(-2*Jy*dt,wires=[q1,q2])
        for i in range(4):
            q1=i
            q2=(i+1)%4
            qml.CNOT(wires=[q1,q2])
            qml.DepolarizingChannel(p, wires=q2)
            qml.RZ(-2*Jz*dt,wires=[q2])
            qml.CNOT(wires=[q1,q2])
            qml.DepolarizingChannel(p, wires=q2)
        for i in range(4):
            qml.RX(-2*h*dt,wires=i)   
    return qml.state()


In [55]:
def calculate_fidelity(couplings, p, time, depth):
    """This function returns the fidelity between the final states of the noisy and
    noiseless Trotterizations of the Heisenberg models, using only CNOT and rotation gates

    Args:
        couplings (list(float)): 
            A list with the J_x, J_y, J_z and h parameters in the Heisenberg Hamiltonian, as
            defined in the problem statement.
        p (float): The depolarization probability of the depolarization gate that acts on the
                   target qubit of each CNOT gate.
        time (float): The period of time evolution simulated by the Trotterization.
        depth (int): The Trotterization depth.

    Returns:
        (float): Fidelity between final states of the noisy and noiseless Trotterizations
    """
    return qml.math.fidelity(heisenberg_trotter(couplings,0,time, depth),heisenberg_trotter(couplings,p,time,depth))

# These functions are responsible for testing the solution.
def run(test_case_input: str) -> str:

    ins = json.loads(test_case_input)
    output =calculate_fidelity(*ins)

    return str(output)

def check(solution_output: str, expected_output: str) -> None:
    """
    Compare solution with expected.

    Args:
            solution_output: The output from an evaluated solution. Will be
            the same type as returned.
            expected_output: The correct result for the test case.

    Raises: 
            ``AssertionError`` if the solution output is incorrect in any way.
            
    """
    def create_hamiltonian(params):

        couplings = [-params[-1]]
        ops = [qml.PauliX(3)]

        for i in range(3):

            couplings = [-params[-1]] + couplings
            ops = [qml.PauliX(i)] + ops        

        for i in range(4):

            couplings = [-params[-2]] + couplings
            ops = [qml.PauliZ(i)@qml.PauliZ((i+1)%4)] + ops

        for i in range(4):

            couplings = [-params[-3]] + couplings
            ops = [qml.PauliY(i)@qml.PauliY((i+1)%4)] + ops

        for i in range(4):

            couplings = [-params[0]] + couplings
            ops = [qml.PauliX(i)@qml.PauliX((i+1)%4)] + ops    

        return qml.Hamiltonian(couplings,ops)

    @qml.qnode(dev)
    def evolve(params, time, depth):

        qml.ApproxTimeEvolution(create_hamiltonian(params), time, depth)

        return qml.state()
    
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    
    tape = heisenberg_trotter.qtape
    names = [op.name for op in tape.operations]
    
    random_params = np.random.uniform(low = 0.8, high = 3.0, size = (4,) )
    
    assert qml.math.fidelity(heisenberg_trotter(random_params,0,1,2),evolve(random_params,1,2)) >= 1, "Your circuit does not Trotterize the Heisenberg Model"
    
    assert names.count('ApproxTimeEvolution') == 0, "Your circuit must not use the built-in PennyLane Trotterization"
     
    assert set(names) == {'DepolarizingChannel', 'RX', 'RY', 'RZ', 'CNOT'}, "Your circuit must only use RX, RY, RZ, CNOT, and depolarizing gates (don't use qml.Rot or Paulis)"
    
    assert solution_output >= expected_output-0.005, "Your fidelity is not high enough. You may be using more CNOT gates than needed"




In [56]:
test_cases = [['[[1,2,1,0.3],0.05,2.5,1]', '0.33723981123369573'], ['[[1,3,2,0.3],0.05,2.5,2]', '0.15411351752086694']]

for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '[[1,2,1,0.3],0.05,2.5,1]'...
Correct!
Running test case 1 with input '[[1,3,2,0.3],0.05,2.5,2]'...
Correct!


In [34]:
heisenberg_trotter([1,1,1,0],0,1,2)

array([[ 0.12464932+7.17199882e-17j,  0.        +0.00000000e+00j,
         0.        +0.00000000e+00j, -0.0199238 -1.83657749e-02j,
         0.        +0.00000000e+00j,  0.02604121+4.06787047e-02j,
        -0.0199238 -1.83657749e-02j,  0.        +0.00000000e+00j,
         0.        +0.00000000e+00j, -0.0199238 -1.83657749e-02j,
         0.02604121+4.06787047e-02j,  0.        +0.00000000e+00j,
        -0.0199238 -1.83657749e-02j,  0.        +0.00000000e+00j,
         0.        +0.00000000e+00j, -0.03049692+3.17141864e-01j],
       [ 0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
         0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
         0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
         0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
         0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
         0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
         0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
         

In [18]:
evolve([1,1,1,0],1,2)

array([[ 3.41980165e-01-2.77555756e-17j, -1.11022302e-16-5.20417043e-17j,
        -5.55111512e-17-5.89805982e-17j,  2.96937367e-02-1.93894431e-02j,
        -1.07552856e-16+6.93889390e-18j,  1.06843607e-01-1.47238399e-02j,
         2.96937367e-02-1.93894431e-02j,  3.64291930e-17+0.00000000e+00j,
        -5.63785130e-17-2.08166817e-17j,  2.96937367e-02-1.93894431e-02j,
         1.06843607e-01-1.47238399e-02j,  2.42861287e-17+4.33680869e-19j,
         2.96937367e-02-1.93894431e-02j,  4.42354486e-17-6.93889390e-17j,
         5.55111512e-17-5.55111512e-17j, -4.13698989e-02+4.41613964e-01j],
       [-5.55111512e-17+6.93889390e-17j,  0.00000000e+00+1.66967135e-17j,
         2.77555756e-17+3.46944695e-18j,  3.81639165e-17+4.51028104e-17j,
        -3.46944695e-18+6.93889390e-18j,  2.77555756e-17+8.67361738e-19j,
        -2.77555756e-17-2.42861287e-17j, -3.81639165e-17+3.46944695e-17j,
        -7.80625564e-18+6.93889390e-18j,  2.08166817e-17-3.12250226e-17j,
         2.77555756e-17+1.82145965e-1

In [50]:
random_params = np.random.uniform(low = 0.8, high = 3.0, size = (4,) )
qml.math.fidelity(heisenberg_trotter([1,1,1,1],0,1,2),evolve([1,1,1,1],1,2))

1.0000000782977188

In [7]:
def create_hamiltonian(params):

    couplings = [-params[-1]]
    ops = [qml.PauliX(3)]

    for i in range(3):

        couplings = [-params[-1]] + couplings
        ops = [qml.PauliX(i)] + ops        

    for i in range(4):

        couplings = [-params[-2]] + couplings
        ops = [qml.PauliZ(i)@qml.PauliZ((i+1)%4)] + ops

    for i in range(4):

        couplings = [-params[-3]] + couplings
        ops = [qml.PauliY(i)@qml.PauliY((i+1)%4)] + ops

    for i in range(4):

        couplings = [-params[0]] + couplings
        ops = [qml.PauliX(i)@qml.PauliX((i+1)%4)] + ops    

    return qml.Hamiltonian(couplings,ops)

@qml.qnode(dev)
def evolve(params, time, depth):

    qml.ApproxTimeEvolution(create_hamiltonian(params), time, depth)

    return qml.state()

In [8]:
qml.math.fidelity(heisenberg_trotter([1,1,1,0],0,1,2),evolve([1,1,1,0],1,2))

1 1 1


0.34198016512607715